In [ ]:
import os
import numpy as np
import glob
import tifffile as tiff

import matplotlib.pyplot as plt

In [ ]:
file_dir = r'sample_data\VJ_OFCVTA_7_260_D6_cropped'
search_path = os.path.join(file_dir,"*.tif")

print(search_path)


The * indicates a "wildcard" that represents any number or type of characters that could be in that specific location in the string

For example the variable `search_path` paired with the glob function below, will search for file paths that start with `file_dir` and end in `.tif`. As long as those two conditions are met, it won't matter what the contents in between are (hence the * wildcard).

Glob is a nice built-in module that finds paths and files based on supplied search parameters. If there are multiple files that satisfy the glob search argument, it will return a list of paths. 

In [ ]:
glob_list = sorted(glob.glob(search_path))
glob_list

In [ ]:
len(glob_list)

Maybe we don't want to load in all the files above. What can we do to filter the list?

In [ ]:
glob_first_50 = glob_list[:50]
print(glob_first_50)

glob_last_50 = glob_list[50:]

In [ ]:
# this is how we get every other file starting from the 0th (even files). 
glob_every_other_even = glob_list[::2]
glob_every_other

# How do we get every odd file?

In [ ]:
first_frame = tiff.imread(glob_every_other_even[0])
first_frame

In [ ]:
first_frame.shape

In [ ]:
plt.imshow(first_frame)

In [ ]:
plt.imshow(tiff.imread(glob_every_other_even[40]))

Let's load up all frames from the glob list.

In [ ]:
data_2p = np.empty([len(glob_every_other_even),142,247])
frame_counter = 0
for frame_path in glob_every_other_even:
    data_2p[frame_counter,:,:] = tiff.imread(frame_path)
    frame_counter += 1 # =+ doesn't work!

In [ ]:
plt.imshow(np.mean(data_2p, axis=0))

In [ ]:
data_2p_mean = np.mean(data_2p, axis=0)

data_2p_mean.argmax()

Exercise: plotting max point on 2p image

** Hint: Researchc and use the function `np.unravel_index()`. You'll also need to make use of the method `.shape`

In [ ]:
max_xy_coord = np.unravel_index(data_2p_mean.argmax(), data_2p_mean.shape) 

plt.imshow(np.mean(data_2p, axis=0))
plt.plot(max_xy_coord[1], max_xy_coord[0], marker="o", markersize=10, markerfacecolor="red")

Cropping/slicing data

In [ ]:
data_2p_cropped = data_2p[:10, 55:80, :]

In [ ]:
plt.imshow(np.mean(data_2p_cropped, axis=0))